In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Into Json Dataset**

In [2]:
import os
import re
import json

root_path = "/content/drive/MyDrive/Rearch_Dimas/NER-DATASET/"

def preprocess(input_path, save_path, mode):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    data_path = os.path.join(save_path, mode + ".json")
    labels = set()
    result = []
    tmp = {}
    tmp['id'] = 0
    tmp['text'] = ''
    tmp['labels'] = []

    with open(input_path, 'r', encoding='utf-8') as fp:
        lines = fp.readlines()
        texts = []
        entities = []
        words = []
        entity_tmp = []
        entities_tmp = []
        for line in lines:
            line = line.strip().split("	")
            if len(line) == 2:
                word = line[0]
                label = line[1]
                words.append(word)

                if "B-" in label:
                    entity_tmp.append(word)
                    if (" ".join(entity_tmp), label.split("-")[-1]) not in entities_tmp:
                        entities_tmp.append(("".join(entity_tmp), label.split("-")[-1]))
                    labels.add(label.split("-")[-1])
                    entity_tmp = []

                elif "I-" in label:
                    entity_tmp.append(word)
                    if (" ".join(entity_tmp), label.split("-")[-1]) not in entities_tmp:
                        entities_tmp.append(("".join(entity_tmp), label.split("-")[-1]))
                    entity_tmp = []
                    labels.add(label.split("-")[-1])
            else:
                texts.append(" ".join(words))
                entities.append(entities_tmp)
                words = []
                entities_tmp = []

    i = 0
    for text, entity in zip(texts, entities):

        if entity:
            ltmp = []
            for ent, type in entity:
                for span in re.finditer(ent, text):
                    start = span.start()
                    end = span.end()
                    ltmp.append((type, start, end, ent))
                    # print(ltmp)
            ltmp = sorted(ltmp, key=lambda x: (x[1], x[2]))
            tmp['id'] = i
            tmp['text'] = text
            for j in range(len(ltmp)):
                tmp['labels'].append(["T{}".format(str(j)), ltmp[j][0], ltmp[j][1], ltmp[j][2], ltmp[j][3]])
        else:
            tmp['id'] = i
            tmp['text'] = text
            tmp['labels'] = []
        result.append(tmp)
        # print(i, text, entity, tmp)
        tmp = {}
        tmp['id'] = 0
        tmp['text'] = ''
        tmp['labels'] = []
        i += 1

    with open(data_path, 'w', encoding='utf-8') as fp:
        fp.write(json.dumps(result, ensure_ascii=False))

    if mode == "ner_dataset":
        label_path = os.path.join(save_path, "labels.json")
        with open(label_path, 'w', encoding='utf-8') as fp:
            fp.write(json.dumps(list(labels), ensure_ascii=False))


preprocess(root_path + "BIO/final-data.txt", root_path + 'excel_data/NER Dataset', "ner_dataset")

labels_path = os.path.join(root_path + 'excel_data/NER Dataset/labels.json')
with open(labels_path, 'r') as fp:
    labels = json.load(fp)

tmp_labels = []
tmp_labels.append('O')
for label in labels:
    tmp_labels.append('B-' + label)
    tmp_labels.append('I-' + label)

label2id = {}
for k, v in enumerate(tmp_labels):
    label2id[v] = k
path = root_path + 'excel_data/NER Dataset/'
if not os.path.exists(path):
    os.makedirs(path)
with open(os.path.join(path, "nor_ent2id.json"), 'w') as fp:
    fp.write(json.dumps(label2id, ensure_ascii=False))

In [3]:
! ls -lh "/content/drive/MyDrive/Rearch_Dimas/NER-DATASET/excel_data/NER Dataset/ner_dataset.json"

-rw------- 1 root root 443K Jul 15 02:49 '/content/drive/MyDrive/Rearch_Dimas/NER-DATASET/excel_data/NER Dataset/ner_dataset.json'


In [4]:
import json

ner_json_dataset = open("/content/drive/MyDrive/Rearch_Dimas/NER-DATASET/excel_data/NER Dataset/ner_dataset.json", "r")
json.load(ner_json_dataset)

[{'id': 0,
  'labels': [['T0', 'plant', 46, 55, 'digitalis'],
   ['T1', 'disease', 64, 75, 'arrhythmias']],
  'text': 'studies on magnesium s mechanism of action in digitalis induced arrhythmias'},
 {'id': 1,
  'labels': [['T0', 'plant', 41, 50, 'digitalis'],
   ['T1', 'disease', 59, 70, 'arrhythmias']],
  'text': 'the mechanism by which magnesium affects digitalis induced arrhythmias was studied in dogs with and without beta receptor'},
 {'id': 2,
  'labels': [['T0', 'plant', 147, 156, 'digitalis'],
   ['T1', 'disease', 163, 174, 'arrhythmias']],
  'text': 'magnesium s direct effect on calcium and potassium fluxes across the myocardial cell membrane may be the mechanism of its antiarrhythmic action in digitalis toxic arrhythmias'},
 {'id': 3,
  'labels': [['T0', 'plant', 21, 26, 'green'],
   ['T1', 'plant', 27, 30, 'tea'],
   ['T2', 'disease', 60, 64, 'skin'],
   ['T3', 'disease', 65, 75, 'papillomas']],
  'text': 'inhibitory effect of green tea on the growth of established skin papil

# **Into Excel Dataset**

In [5]:
! pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 8.0 MB/s 


In [6]:
import json

import pandas as pd
import xlsxwriter
import os
from collections import Counter
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
nltk.download('averaged_perceptron_tagger')

with open('/content/drive/MyDrive/Rearch_Dimas/NER-DATASET/excel_data/NER Dataset/ner_dataset.json', encoding='utf-8') as f:
    raw = json.load(f)

content_1 = []
content_2 = []
for i, item in enumerate(raw):
    # print(i, item)
    text = item["text"]
    # print(text)
    ids = item["id"]
    # print(ids)
    labels = item["labels"]

    labels_tags = []
    labels_word = []
    if len(labels) != 0:
        labels = [[label[0], label[2], label[3], label[4], label[1]] for label in labels]
        labels_word = [label[3] for label in labels]
        labels_tags = [label[4] for label in labels]
    # print(labels)

    texts = item["text"]
    lower_case = texts.lower()
    tokens = nltk.word_tokenize(lower_case)
    tags = nltk.pos_tag(tokens)
    # print(f"tag: {tags}")
    counts = Counter(tag for word, tag in tags)
    # print(f"count: {counts}")

    tags_word = []
    tags_pos = []
    if len(tags) != 0:
        tags = [[pos_tag[0], pos_tag[1]] for pos_tag in tags]
        tags_word = [pos_tag[0] for pos_tag in tags]
        tags_pos = [pos_tag[1] for pos_tag in tags]

    print(tags)

    content_1.append([f"Sentence: {ids}", labels_word, labels_tags])
    content_2.append([f"Sentence: {ids}", tags_word, tags_pos])
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[['studies', 'NNS'], ['on', 'IN'], ['magnesium', 'NN'], ['s', 'NN'], ['mechanism', 'NN'], ['of', 'IN'], ['action', 'NN'], ['in', 'IN'], ['digitalis', 'NN'], ['induced', 'VBD'], ['arrhythmias', 'NNS']]
[['the', 'DT'], ['mechanism', 'NN'], ['by', 'IN'], ['which', 'WDT'], ['magnesium', 'NN'], ['affects', 'VBZ'], ['digitalis', 'RB'], ['induced', 'JJ'], ['arrhythmias', 'NN'], ['was', 'VBD'], ['studied', 'VBN'], ['in', 'IN'], ['dogs', 'NNS'], ['with', 'IN'], ['and', 'CC'], ['without', 'IN'], ['beta', 'NN'], ['receptor', 'NN']]
[['magnesium', 'NN'], ['s', 'NN'], ['direct', 'JJ'], ['effect', 'NN'], ['on', 'IN'], ['calcium', 'NN'], ['and', 'CC'], ['potassium', 'NN'], ['fluxes', 'NNS'], ['across', 'IN'], ['the', 'DT'], ['myocardial', 'NN'], ['cell', 'NN'], ['membrane', 'NN'], ['may', 'MD'], ['be', 'VB'], ['the', 'DT'], ['mechanism', 'NN'], ['of', 'IN'], ['its', 'PRP$'], ['antiarrhythmic', 'JJ'], ['action', 'NN'], ['in', 'IN'], ['digitalis', 'NN'], ['toxic', 'NN'], ['arrhythmias', 'NN']]
[['inhib

In [7]:
df_labels = pd.DataFrame(content_1, columns=['Sentence #','Word', 'Tag']).explode(['Word', 'Tag'])

In [8]:
data_labels = df_labels.drop_duplicates(['Sentence #','Word','Tag'], keep='first')

In [9]:
data_labels.head()

,Sentence #,Word,Tag
0,Sentence: 0,digitalis,plant
0,Sentence: 0,arrhythmias,disease
1,Sentence: 1,digitalis,plant
1,Sentence: 1,arrhythmias,disease
2,Sentence: 2,digitalis,plant


In [10]:
data_labels.tail()

,Sentence #,Word,Tag
1307,Sentence: 1307,psychosis,disease
1308,Sentence: 1308,schizophrenia,disease
1308,Sentence: 1308,cannabis,plant
1309,Sentence: 1309,cannabis,plant
1309,Sentence: 1309,schizophrenia,disease


In [11]:
data_labels.shape

(3734, 3)

In [12]:
df_tags = pd.DataFrame(content_2, columns=['Sentence #', 'Word', 'POS']).explode(['Word', 'POS'])

In [13]:
data_tags = df_tags.drop_duplicates(['Sentence #','Word','POS'], keep='first')

In [14]:
data_tags.head()

,Sentence #,Word,POS
0,Sentence: 0,studies,NNS
0,Sentence: 0,on,IN
0,Sentence: 0,magnesium,NN
0,Sentence: 0,s,NN
0,Sentence: 0,mechanism,NN


In [15]:
data_tags.tail()

,Sentence #,Word,POS
1309,Sentence: 1309,is,VBZ
1309,Sentence: 1309,due,JJ
1309,Sentence: 1309,to,TO
1309,Sentence: 1309,shared,VBN
1309,Sentence: 1309,genetic,JJ


In [16]:
data_tags.shape

(31694, 3)

In [17]:
join_data = pd.merge(data_tags, data_labels, how='left', on=['Sentence #', 'Word'])

In [18]:
join_data = join_data.fillna("O")

In [19]:
join_data = join_data.drop_duplicates(['Sentence #','Word','POS', 'Tag'], keep='first')

In [20]:
join_data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 0,studies,NNS,O
1,Sentence: 0,on,IN,O
2,Sentence: 0,magnesium,NN,O
3,Sentence: 0,s,NN,O
4,Sentence: 0,mechanism,NN,O


In [21]:
join_data.tail()

,Sentence #,Word,POS,Tag
31689,Sentence: 1309,is,VBZ,O
31690,Sentence: 1309,due,JJ,O
31691,Sentence: 1309,to,TO,O
31692,Sentence: 1309,shared,VBN,O
31693,Sentence: 1309,genetic,JJ,O


In [22]:
join_data.shape

(31694, 4)

In [23]:
join_data.describe()

,Sentence #,Word,POS,Tag
count,31694,31694,31694,31694
unique,1310,3089,32,3
top,Sentence: 267,and,NN,O
freq,101,930,9763,27882


In [24]:
join_data.to_excel('/content/drive/MyDrive/Rearch_Dimas/NER-DATASET/excel_data/NER Dataset/ner_dataset.xlsx', index=False)

# **Into CSV Dataset**

In [25]:
join_data.to_csv('/content/drive/MyDrive/Rearch_Dimas/NER-DATASET/csv_data/ner_dataset.csv', index=None, header=True)

# **Preview**

In [26]:
join_data.head(20)

,Sentence #,Word,POS,Tag
0,Sentence: 0,studies,NNS,O
1,Sentence: 0,on,IN,O
2,Sentence: 0,magnesium,NN,O
3,Sentence: 0,s,NN,O
4,Sentence: 0,mechanism,NN,O
5,Sentence: 0,of,IN,O
6,Sentence: 0,action,NN,O
7,Sentence: 0,in,IN,O
8,Sentence: 0,digitalis,NN,plant
9,Sentence: 0,induced,VBD,O


In [27]:
join_data.tail(20)

,Sentence #,Word,POS,Tag
31674,Sentence: 1309,of,IN,O
31675,Sentence: 1309,the,DT,O
31676,Sentence: 1309,variance,NN,O
31677,Sentence: 1309,in,IN,O
31678,Sentence: 1309,cannabis,NN,plant
31679,Sentence: 1309,use,NN,O
31680,Sentence: 1309,these,DT,O
31681,Sentence: 1309,findings,NNS,O
31682,Sentence: 1309,suggest,VBP,O
31683,Sentence: 1309,that,IN,O
